# 下载数据集，并对数据集进行处理！

### 1. 下载数据集

In [1]:
from datasets import load_dataset

dataset_path = "/raid/gfc/llm/datasets/Chinese-medical-dialogue"
ds = load_dataset("ticoAg/Chinese-medical-dialogue", cache_dir=dataset_path)

# 查看数据集的基本信息
print("数据集结构：")
for k, v in ds.items():
    print(f"{k}: {v}")

# 获取数据集的大小
print(f"\n训练集大小: {len(ds['train'])}")

# 查看数据集的列名（特征）
print("\n数据集的特征：")
print(ds['train'].features)

/raid/gfc/anaconda/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


数据集结构：
train: Dataset({
    features: ['instruction', 'input', 'output', 'history'],
    num_rows: 799743
})

训练集大小: 799743

数据集的特征：
{'instruction': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None), 'history': Value(dtype='null', id=None)}


### 2. 清洗数据集

##### 2.1 缺失值处理

In [2]:
# 数据清洗
def is_valid(example):
    # 三项都不能为空且不是纯空格
    return all([
        example['instruction'] and example['instruction'].strip(),
        example['input'] and example['input'].strip(),
        example['output'] and example['output'].strip()
    ])

dataset = ds['train']
print(f"原始样本数: {len(dataset)}")

# 过滤空值
ds_clean = dataset.filter(is_valid)
print(f"清洗后样本数: {len(ds_clean)}")

print(ds_clean[0])
print(ds_clean[0]['instruction'])
print(type(ds_clean))

原始样本数: 799743
清洗后样本数: 799736
{'instruction': '小儿肥胖超重该如何治疗', 'input': '女宝宝，刚7岁，这一年，察觉到，我家孩子身上肉很多，而且，食量非常的大，平时都不喜欢吃去玩，请问：小儿肥胖超重该如何治疗。', 'output': '孩子出现肥胖症的情况。家长要通过孩子运功和健康的饮食来缓解他的症状，可以先让他做一些有氧运动，比如慢跑，爬坡，游泳等，并且饮食上孩子多吃黄瓜，胡萝卜，菠菜等，禁止孩子吃一些油炸食品和干果类食物，这些都是干热量高脂肪的食物，而且不要让孩子总是吃完就躺在床上不动，家长在治疗小儿肥胖期间如果孩子情况严重就要及时去医院在医生的指导下给孩子治疗。', 'history': None}
小儿肥胖超重该如何治疗
<class 'datasets.arrow_dataset.Dataset'>


##### 2.2 格式化规范
去除多余空格、特殊符号。
统一全角/半角、简繁体（如有需要）。
统一标点符号。

In [3]:
import re
import jaconv
from zhconv import convert

def normalize_text(text):
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9，。！？、；：“”‘’（）《》【】]', '', text)
    text = jaconv.z2h(text, kana=False, ascii=True, digit=True)
    text = convert(text, 'zh-cn')
    return text

def normalize_example(example):
    for col in ['instruction', 'input', 'output']:
        example[col] = normalize_text(str(example[col]))
    return example

# 推荐：直接在 HuggingFace Dataset 上并行处理
ds_clean = ds_clean.map(normalize_example, num_proc=4)  # 可根据CPU核数调整num_proc

print("数据格式化完成！")

数据格式化完成！


##### 2.3 去除冗余数据集

In [4]:
# 去重（以instruction+input+output为唯一标识）
import pandas as pd
from simhash import Simhash

ds_clean = ds_clean.to_pandas()
ds_clean = ds_clean.drop_duplicates(subset=['instruction', 'input', 'output'])
print(f"去重后样本数: {len(ds_clean)}")

# # 近似去重示例（SimHash）
# def get_simhash(text):
#     return Simhash(text).value

# ds_clean['simhash'] = ds_clean.apply(lambda row: get_simhash(row['instruction'] + row['input'] + row['output']), axis=1)
# ds_clean = ds_clean.drop_duplicates(subset=['simhash'])
# print(f"近似去重后样本数: {len(ds_clean)}")

去重后样本数: 752436


In [5]:
import os

# 存储格式化以后的数据集
formated_dataset_path = os.path.join(dataset_path, "formatted_dataset.csv")

ds_clean.to_csv(formated_dataset_path, index=False, encoding='utf-8')
print(f"格式化后的数据集已保存到: {formated_dataset_path}")

格式化后的数据集已保存到: /raid/gfc/llm/datasets/Chinese-medical-dialogue/formatted_dataset.csv


##### 2.4 异常样本过滤

In [6]:
# # 可选：过滤过短/过长
# ds_clean = pd.read_csv(formated_dataset_path, encoding='utf-8')

# # 拼接instruction和input
# ds_clean['prompt'] = ds_clean['instruction'].astype(str) + ds_clean['input'].astype(str)

# # exceptions_idx = ds_clean[ds_clean['prompt'].str.len() < 5].index
# # print(ds_clean.iloc[exceptions_idx[0]])
# # 过滤拼接后长度过短的样本
# ds_clean = ds_clean[ds_clean['prompt'].str.len() >= 5]

# print(f"过滤过短/过长后的样本数: {len(ds_clean)}")

# # 保存清洗后的数据
# cleaned_dataset_path = os.path.join(dataset_path, "cleaned_dataset.csv")
# ds_clean.to_csv(cleaned_dataset_path, index=False, encoding='utf-8')

##### 2.4 语义级别去重

In [7]:
from transformers import BertModel, BertTokenizer
import torch
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
bert_cache_dir = "/raid/gfc/llm/models"
bert_model = BertModel.from_pretrained("bert-base-chinese", cache_dir=bert_cache_dir).to(device)
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese", cache_dir=bert_cache_dir)

print("bert模型加载完成！")
# print(bert_model)

bert模型加载完成！


In [ ]:
# 语义级别去重
from torch.utils.data import DataLoader
import faiss

# 1. 读取数据
df = pd.read_csv(formated_dataset_path, encoding='utf-8')

# 2. 拼接文本
texts = (df['instruction'].astype(str) + '[SEP]' + df['input'].astype(str) + '[SEP]' + df['output'].astype(str)).tolist()
# texts = texts[:100]  # 测试，仅处理前100条数据，实际使用时可去掉此行
# for text in texts:
#     print(text)

def batch_get_bert_embedding(texts, tokenizer, model, device, batch_size=64):
    all_embeds = []
    loader = DataLoader(texts, batch_size=batch_size, shuffle=False)
    for batch in tqdm(loader, desc="BERT批量提取嵌入"):
        inputs = tokenizer(
            list(batch), 
            return_tensors="pt", 
            truncation=True, 
            max_length=512, 
            padding=True
        )
        # dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
        inputs = {k: v.to(device) for k, v in inputs.items()}
        # print(inputs['input_ids'].shape)  # [batch_size, seq_len]
        with torch.no_grad():
            outputs = model(**inputs)
            # print(outputs.keys()) # dict_keys(['last_hidden_state', 'pooler_output'])
            # print(outputs.last_hidden_state.shape)  # [batch_size, seq_len, hidden_size]
            # print(outputs.pooler_output.shape)  # [batch_size, hidden_size]
            # print(outputs.pooler_output.shape == outputs.last_hidden_state[:, 0, :].shape) # True
            # print(outputs.pooler_output == outputs.last_hidden_state[:, 0, :]) # False
            # 取[CLS]的输出作为句子嵌入
            embeds = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            all_embeds.append(embeds)
        
    return np.vstack(all_embeds)

# 使用方法
embeddings = batch_get_bert_embedding(texts, tokenizer, bert_model, device, batch_size=256)

# 存储embddings
embeddings_path = os.path.join(dataset_path, "embeddings.npy")
np.save(embeddings_path, embeddings)
print(f"嵌入已保存到: {embeddings_path}")

In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')
def keybert_keywords(row, topk=5):
    text = row['instruction'] + row['input'] + row['output']
    keywords = kw_model.extract_keywords(text, top_n=topk)
    return [kw[0] for kw in keywords]

df['keywords'] = df.apply(keybert_keywords, axis=1)

                  instruction  \
70000       请问孩子出生2个多月稀大便怎么办?   
70001  拉肚子瞧了好几回都是好两天还拉好两天怎么办?   
70002         宝宝七个月一直拉肚子怎么治疗?   
70003      我宝宝两周岁了腿还软不能走路怎么办?   
70004          孩子老清嗓子还痰多怎么回事?   
70005            入睡后汗特别多怎么办呢?   
70006        五岁的宝宝鼻子总爱出血怎么办呢?   
70007           7岁的男孩会有可能手淫吗?   
70008            我儿子是否得了病毒感染?   
70009           小儿癫痫要做哪些仔细检查?   

                                                   input  \
70000  您好孩子降生一个多月后到现在2个多月一直是35解一次稀大便,放屁多又臭吃得少,昼夜一惊一诈怎么办?   
70001  宝宝这几天一直拉肚子瞧了好几回都是好两天还拉好两天还拉,吃晚饭挺正常的也玩就是拉肚子,一天两...   
70002  宝宝七个月!一直拉肚子后就换了乳酸菌奶粉?吃后不拉拉!不久后又拉白色粑粑和绿色的!有时还吃啥...   
70003  两周岁了腿软不能够走路!下地学走路一直是点起脚尖走!曾经的治疗情况和效果:无在乎怎样的帮助:...   
70004  孩子老是清嗓子,不是发烧,是不是抽动症啊。河南郑州哪里可以治疗吗?郑州建设东路24那个医院怎...   
70005  宝宝入眠治好汗特别多,每次都是,现在天气不是很热,宝宝穿的衣服也并不多,怎么会出这么多汗呢,...   
70006  我的孩子五岁了,经常爱流鼻血,流鼻血成了我孩子的家常便饭,他爱用手去抠鼻子,也不知晓他的鼻子...   
70007  近来发觉孩子一直往卫生间跑,在里面一呆就是好长时间。一次偷偷看一看他在干什么。发觉他正用手在...   
70008  孩子是从前几天已经开始不舒服的,从幼儿园回去以后,就有点不舒服,晚上已经开始发高烧,吃了退烧...   
70009  朋友亲戚的孩子,出现过几次癫痫症状,有时活动正在进行时,她

In [ ]:
# 5. 语义去重（两两余弦相似度，阈值可调，建议0.95~0.98）
import faiss

embeddings_path = os.path.join(dataset_path, "embeddings.npy")
embeddings = np.load(embeddings_path)
print(type(embeddings))  # <class 'numpy.ndarray'>
print(f"embeddings从{embeddings_path}加载完成！")

# 假设embeddings为np.ndarray, shape=(N, D)
faiss.normalize_L2(embeddings)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)
D, I = index.search(embeddings, k=10)  # k为每个向量查找的近邻数

# 根据相似度阈值筛选重复样本
threshold = 0.97
visited = set()
keep_idx = []
for i in tqdm(len(embeddings)):
    if i in visited:
        continue
    keep_idx.append(i)
    for j, sim in zip(I[i], D[i]):
        if j != i and sim > threshold:
            visited.add(j)

df_semantic_dedup = df.iloc[keep_idx].reset_index(drop=True)
print(f"语义去重后样本数: {len(df_semantic_dedup)}")

# 6. 保存
df_semantic_dedup.to_csv(dataset_path + "/semantic_dedup.csv", index=False, encoding='utf-8')
print("语义去重后的数据已保存。")

<class 'numpy.ndarray'>
embeddings从/raid/gfc/llm/datasets/Chinese-medical-dialogue/embeddings.npy加载完成！


### 3. 处理数据集 转换成qwen指令微调的格式

In [20]:
# 处理数据集成qwen微调所需的格式
import os
import numpy as np
import pandas as pd

dataset_path = "/raid/gfc/llm/datasets/Chinese-medical-dialogue"

formatted_dataset_path = os.path.join(dataset_path, "formatted_dataset.csv")
dedup_idx_path = os.path.join(dataset_path, "dedup_idx.npy")
formatted_dataset = pd.read_csv(formatted_dataset_path, encoding='utf-8')
dataset_idx = np.load(dedup_idx_path)
dataset = formatted_dataset.iloc[dataset_idx].reset_index(drop=True)
print(f"一共{len(dataset)}条数据")

一共593437条数据


In [21]:
from sklearn.model_selection import train_test_split

df = dataset
# 打乱数据
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# 先划分训练集和临时集
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
# 再将临时集一分为二，得验证集和测试集
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"训练集: {len(train_df)}, 验证集: {len(val_df)}, 测试集: {len(test_df)}")

训练集: 474749, 验证集: 59344, 测试集: 59344


In [22]:
import json

def to_prompt_response(row):
    prompt = str(row["instruction"]) + "\n" + str(row["input"])
    response = str(row["output"])
    return {"prompt": prompt, "response": response}

def save_jsonl(df, path):
    with open(path, "w", encoding="utf-8") as f:
        for _, row in df.iterrows():
            item = to_prompt_response(row)
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

save_jsonl(train_df, f"{dataset_path}/train.jsonl")
save_jsonl(val_df, f"{dataset_path}/val.jsonl")
save_jsonl(test_df, f"{dataset_path}/test.jsonl")
print("数据集已保存为jsonl格式。")

数据集已保存为jsonl格式。
